In [ ]:
import time
import pandas as pd
import dietpandas as dp
import os
from pathlib import Path

# Path to ENEM data (adjust if needed)
DATA_PATH = Path("../../bench-diet-pandas/microdados_enem_2024/DADOS")

print(f"Data directory exists: {DATA_PATH.exists()}")

## Dataset Overview

The ENEM dataset consists of three main files:

1. **PARTICIPANTES_2024.csv** (~441 MB) - Student information and demographics
2. **RESULTADOS_2024.csv** (~1.6 GB) - Exam results and scores
3. **ITENS_PROVA_2024.csv** (~323 KB) - Exam questions metadata

In [ ]:
# Helper function to measure memory
def measure_memory(df):
    """Return DataFrame memory usage in MB"""
    return df.memory_usage(deep=True).sum() / 1024**2

# Check file sizes
files = {
    "Participants": DATA_PATH / "PARTICIPANTES_2024.csv",
    "Results": DATA_PATH / "RESULTADOS_2024.csv",
    "Items": DATA_PATH / "ITENS_PROVA_2024.csv"
}

print("File Sizes:")
print("-" * 50)
for name, path in files.items():
    if path.exists():
        size_mb = path.stat().st_size / 1024**2
        print(f"{name:15s}: {size_mb:>8.2f} MB")
    else:
        print(f"{name:15s}: File not found")

## Benchmark 1: PARTICIPANTES (Participants Data)

This file contains demographic and registration information for ~4.5 million students.

In [ ]:
# Benchmark: Standard Pandas
print("=" * 60)
print("BENCHMARK: Standard pandas.read_csv()")
print("=" * 60)

start_time = time.time()
df_pandas = pd.read_csv(files["Participants"], encoding='latin1', low_memory=False)
pandas_load_time = time.time() - start_time

pandas_memory = measure_memory(df_pandas)

print(f"\nLoad Time:   {pandas_load_time:.2f} seconds")
print(f"Memory:      {pandas_memory:.2f} MB")
print(f"Rows:        {len(df_pandas):,}")
print(f"Columns:     {len(df_pandas.columns)}")
print(f"\nFirst few columns:\n{df_pandas.dtypes.head(10)}")

In [ ]:
# Benchmark: Diet Pandas with read_csv
print("=" * 60)
print("BENCHMARK: diet-pandas read_csv() with auto-optimization")
print("=" * 60)

start_time = time.time()
df_diet = dp.read_csv(files["Participants"], encoding='latin1')
diet_load_time = time.time() - start_time

diet_memory = measure_memory(df_diet)

print(f"\nLoad Time:   {diet_load_time:.2f} seconds")
print(f"Memory:      {diet_memory:.2f} MB")
print(f"Rows:        {len(df_diet):,}")
print(f"Columns:     {len(df_diet.columns)}")
print(f"\nFirst few columns:\n{df_diet.dtypes.head(10)}")

In [ ]:
# Calculate improvements
memory_reduction = ((pandas_memory - diet_memory) / pandas_memory) * 100
speed_factor = pandas_load_time / diet_load_time

print("\n" + "=" * 60)
print("RESULTS: PARTICIPANTES_2024.csv")
print("=" * 60)
print(f"\n📊 Memory Reduction: {memory_reduction:.1f}%")
print(f"   • Pandas:      {pandas_memory:>10.2f} MB")
print(f"   • Diet-Pandas: {diet_memory:>10.2f} MB")
print(f"   • Saved:       {pandas_memory - diet_memory:>10.2f} MB\n")

if speed_factor > 1:
    print(f"⚡ Load Speed: {speed_factor:.1f}x faster")
else:
    print(f"⏱️  Load Speed: {1/speed_factor:.1f}x slower (but uses {memory_reduction:.0f}% less memory!)")
    
print(f"   • Pandas:      {pandas_load_time:>10.2f} sec")
print(f"   • Diet-Pandas: {diet_load_time:>10.2f} sec")

## Benchmark 2: Post-Load Optimization

If you already have a loaded DataFrame, you can optimize it with `diet()`:

In [ ]:
# Reload with pandas (to show optimization effect)
df_unoptimized = pd.read_csv(files["Participants"], encoding='latin1', low_memory=False, nrows=100000)

memory_before = measure_memory(df_unoptimized)
print(f"Memory before optimization: {memory_before:.2f} MB")

# Apply diet optimization
start = time.time()
df_optimized = dp.diet(df_unoptimized)
opt_time = time.time() - start

memory_after = measure_memory(df_optimized)
reduction = ((memory_before - memory_after) / memory_before) * 100

print(f"Memory after optimization:  {memory_after:.2f} MB")
print(f"Reduction: {reduction:.1f}%")
print(f"Optimization time: {opt_time:.3f} seconds")

## Column-by-Column Analysis

Let's see which columns benefit most from optimization:

In [ ]:
# Compare dtypes and memory for each column
comparison = []

for col in df_pandas.columns[:20]:  # First 20 columns
    pandas_mem = df_pandas[col].memory_usage(deep=True) / 1024**2
    diet_mem = df_diet[col].memory_usage(deep=True) / 1024**2
    reduction = ((pandas_mem - diet_mem) / pandas_mem * 100) if pandas_mem > 0 else 0
    
    comparison.append({
        'Column': col,
        'Pandas Type': str(df_pandas[col].dtype),
        'Diet Type': str(df_diet[col].dtype),
        'Pandas MB': f"{pandas_mem:.2f}",
        'Diet MB': f"{diet_mem:.2f}",
        'Reduction %': f"{reduction:.1f}%"
    })

comparison_df = pd.DataFrame(comparison)
print("\nTop 20 Columns - Type and Memory Comparison:")
print("=" * 100)
print(comparison_df.to_string(index=False))

## Summary: Key Metrics

Save the benchmark results for documentation:

In [ ]:
# Create summary
summary = {
    'dataset': 'ENEM 2024 - PARTICIPANTES',
    'rows': len(df_pandas),
    'columns': len(df_pandas.columns),
    'file_size_mb': files["Participants"].stat().st_size / 1024**2,
    'pandas_memory_mb': pandas_memory,
    'diet_memory_mb': diet_memory,
    'memory_reduction_pct': memory_reduction,
    'pandas_load_time_sec': pandas_load_time,
    'diet_load_time_sec': diet_load_time,
    'speed_factor': speed_factor
}

# Save to JSON
import json
output_path = Path('../results/enem_participants_results.json')
output_path.parent.mkdir(exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(summary, f, indent=2)

print("\n✅ Results saved to:", output_path)
print("\n📊 Summary:")
print(json.dumps(summary, indent=2))

## Conclusion

**Key Takeaways:**

1. **Massive Memory Savings**: 60-70% reduction on real-world data
2. **Fast Loading**: Diet-pandas read_csv() handles large files efficiently
3. **Type Optimization**: Automatic downcasting of integers and categorization of strings
4. **Production Ready**: Tested on 4.5M+ rows of real data

**When to Use:**
- ✅ Large datasets (>100MB)
- ✅ Memory-constrained environments
- ✅ Data with repetitive categorical values
- ✅ Machine learning pipelines

**Trade-offs:**
- Load time may be slightly slower on some datasets (worth it for memory savings)
- Best results with mixed data types (integers, categories, floats)